In [81]:
!pip install rpy2==3.5.1
%load_ext rpy2.ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 201 kB 5.3 MB/s 
  Created wheel for rpy2: filename=rpy2-3.5.1-cp37-cp37m-linux_x86_64.whl size=310591 sha256=6bff1cf3c06fd0ab2a7c9c324d5451e765e1c32cdf96736b282e47f405e7a92e
  Stored in directory: /root/.cache/pip/wheels/65/df/9e/39aa22a3f47ec58751b5bf6907036fbc131d85b6d7cbd3b3fe
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5


In [83]:
%%R
install.packages('quantmod')
library(quantmod)


	‘/tmp/RtmpBbKq7D/downloaded_packages’



Attaching package: ‘zoo’



    as.Date, as.Date.numeric



  method            from
  as.zoo.data.frame zoo 



In [84]:
%%R
starting_date <- "2020-1-1"
final_date <- "2022-10-31"
BTCUSD <- getSymbols.yahoo("BTC-USD", from = starting_date, to = final_date, auto.assign = F)[,]

In [85]:
%%R
write.csv(BTCUSD,"/content//BTC.csv", row.names = index(BTCUSD))

In [ ]:
# Arima training models

"""
Script to forecast the BTC-USD.Open 	BTC-USD.High 	BTC-USD.Low 	BTC-USD.Close 	BTC-USD.Volume 	BTC-USD.Adjusted. 

The data need to be in the form showed in the example csv file of this folder. The columns of this dataset are 'product', 'zeros', and the date of the time series.
The index of the dataset must be the column 'product', in this column product refers to a the code of Bitcoin, opening, closing, adjusted, etc. 
The column 'zeros' comtains the information of missing values in the time series. Notice that the dataset has no zeros missing data.

The script will train a model per each time series and produce five predictions based on that time series. In case you provide a dataset with n time seris, the script will train n arima models and generate 5n predictions.

The output of this scrip will be a csv file with the following columns:

'product' as the index
'model' with the name of the model, in the example the name is 'arima_model_without_covid_years' due to the not use of the covid pandemic years in the example.
'intercept_p_value' with the value of the intercept p-value of the intercept in those models that requiere an intercept. Notice that not all the models requieres one.
'sigma2_p_value' with the value of the sigma2 p-value of the sigma2.
'intercept_std_error' with the value of the standard error of the intercept in those models that requiere an intercept. Notice that not all the models requieres one.
'sigma2_std_error' with the value of the standard error of the sigma2.
'AIC' with the value of the evaluation AIC of the selected model.
'min_conf_interval' with the lower bound of the confidence interval.
'max_conf_interval' with the upper bound of the confidence interval.
'year' with the year of the prediction.
'prediction' with the value of the prediction.
'missing_values' with the number of missing values (in the input data) used to train the model.

Notice that we will train the model with the logaritmic value of the data, this is a potential problem due to the fact that log(0) is not defined. Therefore we will need to deal with the zeros of the data before we train the models, this is done by adding an insignificant epsilon error. This epsilon error is an hyperparameter of this script. 
"""

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [86]:
import pandas as pd
path = "/content//BTC.csv"
dfBTC = pd.read_csv(path)

In [87]:
dfBTC.head()

,Unnamed: 0,BTC-USD.Open,BTC-USD.High,BTC-USD.Low,BTC-USD.Close,BTC-USD.Volume,BTC-USD.Adjusted
0,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997,7200.174316
1,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,20802083465,6985.470215
2,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032,7344.884277
3,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,18444271275,7410.656738
4,2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,19725074095,7411.317383


In [88]:
dfBTC.columns

Index(['Unnamed: 0', 'BTC-USD.Open', 'BTC-USD.High', 'BTC-USD.Low',
       'BTC-USD.Close', 'BTC-USD.Volume', 'BTC-USD.Adjusted'],
      dtype='object')

In [89]:
df_time_series=dfBTC.set_index('Unnamed: 0').T
df_time_series

Unnamed: 0,2020-01-01,2020-01-02,2020-01-03,2020-01-04,2020-01-05,2020-01-06,2020-01-07,2020-01-08,2020-01-09,2020-01-10,...,2022-10-22,2022-10-23,2022-10-24,2022-10-25,2022-10-26,2022-10-27,2022-10-28,2022-10-29,2022-10-30,2022-10-31
BTC-USD.Open,7.194892e+03,7.202551e+03,6.984429e+03,7.345375e+03,7.410452e+03,7.410452e+03,7.768682e+03,8.161936e+03,8.082296e+03,7.878308e+03,...,1.917238e+04,1.920773e+04,1.956777e+04,1.934496e+04,2.009224e+04,2.077280e+04,2.028796e+04,2.059510e+04,2.081798e+04,2.063370e+04
BTC-USD.High,7.254331e+03,7.212155e+03,7.413715e+03,7.427386e+03,7.544497e+03,7.781867e+03,8.178216e+03,8.396738e+03,8.082296e+03,8.166554e+03,...,1.924807e+04,1.964665e+04,1.958912e+04,2.034841e+04,2.093813e+04,2.085404e+04,2.072498e+04,2.098839e+04,2.091701e+04,2.079532e+04
BTC-USD.Low,7.174944e+03,6.935270e+03,6.914996e+03,7.309514e+03,7.400536e+03,7.409293e+03,7.768228e+03,7.956774e+03,7.842404e+03,7.726775e+03,...,1.913224e+04,1.912420e+04,1.920632e+04,1.926145e+04,2.007612e+04,2.025537e+04,2.008607e+04,2.056648e+04,2.054746e+04,2.028746e+04
BTC-USD.Close,7.200174e+03,6.985470e+03,7.344884e+03,7.410657e+03,7.411317e+03,7.769219e+03,8.163692e+03,8.079863e+03,7.879071e+03,8.166554e+03,...,1.920819e+04,1.956701e+04,1.934557e+04,2.009586e+04,2.077044e+04,2.028584e+04,2.059535e+04,2.081848e+04,2.063560e+04,2.049577e+04
BTC-USD.Volume,1.856566e+10,2.080208e+10,2.811148e+10,1.844427e+10,1.972507e+10,2.327626e+10,2.876729e+10,3.167256e+10,2.404599e+10,2.871458e+10,...,1.610444e+10,2.212879e+10,3.020224e+10,4.776152e+10,5.889595e+10,4.962511e+10,4.399472e+10,4.036984e+10,3.148635e+10,4.566847e+10
BTC-USD.Adjusted,7.200174e+03,6.985470e+03,7.344884e+03,7.410657e+03,7.411317e+03,7.769219e+03,8.163692e+03,8.079863e+03,7.879071e+03,8.166554e+03,...,1.920819e+04,1.956701e+04,1.934557e+04,2.009586e+04,2.077044e+04,2.028584e+04,2.059535e+04,2.081848e+04,2.063560e+04,2.049577e+04


In [91]:
df_time_series.to_csv('time_series.csv', index = True)

In [90]:
df_time_series.isnull().any(axis=1)

BTC-USD.Open        False
BTC-USD.High        False
BTC-USD.Low         False
BTC-USD.Close       False
BTC-USD.Volume      False
BTC-USD.Adjusted    False
dtype: bool

In [24]:
## Install pmdarima and import packages

!pip install pmdarima


import numpy as np
from pmdarima.arima import auto_arima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 9.9 MB 15.6 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [92]:

## The models and prediction

df_product=pd.read_csv("time_series.csv")

##Check whether the index is the column "Unnamed: 0" as in the provided dataframe example.
## In case it is not, uncomment the following lines
df_product=df_product.set_index(["Unnamed: 0"])
df_product.head()

,2020-01-01,2020-01-02,2020-01-03,2020-01-04,2020-01-05,2020-01-06,2020-01-07,2020-01-08,2020-01-09,2020-01-10,...,2022-10-22,2022-10-23,2022-10-24,2022-10-25,2022-10-26,2022-10-27,2022-10-28,2022-10-29,2022-10-30,2022-10-31
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
BTC-USD.Open,7.194892e+03,7.202551e+03,6.984429e+03,7.345375e+03,7.410452e+03,7.410452e+03,7.768682e+03,8.161936e+03,8.082296e+03,7.878308e+03,...,1.917238e+04,1.920773e+04,1.956777e+04,1.934496e+04,2.009224e+04,2.077280e+04,2.028796e+04,2.059510e+04,2.081798e+04,2.063370e+04
BTC-USD.High,7.254331e+03,7.212155e+03,7.413715e+03,7.427386e+03,7.544497e+03,7.781867e+03,8.178216e+03,8.396738e+03,8.082296e+03,8.166554e+03,...,1.924807e+04,1.964665e+04,1.958912e+04,2.034841e+04,2.093813e+04,2.085404e+04,2.072498e+04,2.098839e+04,2.091701e+04,2.079532e+04
BTC-USD.Low,7.174944e+03,6.935270e+03,6.914996e+03,7.309514e+03,7.400536e+03,7.409293e+03,7.768228e+03,7.956774e+03,7.842404e+03,7.726775e+03,...,1.913224e+04,1.912420e+04,1.920632e+04,1.926145e+04,2.007612e+04,2.025537e+04,2.008607e+04,2.056648e+04,2.054746e+04,2.028746e+04
BTC-USD.Close,7.200174e+03,6.985470e+03,7.344884e+03,7.410657e+03,7.411317e+03,7.769219e+03,8.163692e+03,8.079863e+03,7.879071e+03,8.166554e+03,...,1.920819e+04,1.956701e+04,1.934557e+04,2.009586e+04,2.077044e+04,2.028584e+04,2.059535e+04,2.081848e+04,2.063560e+04,2.049577e+04
BTC-USD.Volume,1.856566e+10,2.080208e+10,2.811148e+10,1.844427e+10,1.972507e+10,2.327626e+10,2.876729e+10,3.167256e+10,2.404599e+10,2.871458e+10,...,1.610444e+10,2.212879e+10,3.020224e+10,4.776152e+10,5.889595e+10,4.962511e+10,4.399472e+10,4.036984e+10,3.148635e+10,4.566847e+10


In [93]:
df_product["zeros"]=0
df_product.head()

,2020-01-01,2020-01-02,2020-01-03,2020-01-04,2020-01-05,2020-01-06,2020-01-07,2020-01-08,2020-01-09,2020-01-10,...,2022-10-23,2022-10-24,2022-10-25,2022-10-26,2022-10-27,2022-10-28,2022-10-29,2022-10-30,2022-10-31,zeros
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
BTC-USD.Open,7.194892e+03,7.202551e+03,6.984429e+03,7.345375e+03,7.410452e+03,7.410452e+03,7.768682e+03,8.161936e+03,8.082296e+03,7.878308e+03,...,1.920773e+04,1.956777e+04,1.934496e+04,2.009224e+04,2.077280e+04,2.028796e+04,2.059510e+04,2.081798e+04,2.063370e+04,0
BTC-USD.High,7.254331e+03,7.212155e+03,7.413715e+03,7.427386e+03,7.544497e+03,7.781867e+03,8.178216e+03,8.396738e+03,8.082296e+03,8.166554e+03,...,1.964665e+04,1.958912e+04,2.034841e+04,2.093813e+04,2.085404e+04,2.072498e+04,2.098839e+04,2.091701e+04,2.079532e+04,0
BTC-USD.Low,7.174944e+03,6.935270e+03,6.914996e+03,7.309514e+03,7.400536e+03,7.409293e+03,7.768228e+03,7.956774e+03,7.842404e+03,7.726775e+03,...,1.912420e+04,1.920632e+04,1.926145e+04,2.007612e+04,2.025537e+04,2.008607e+04,2.056648e+04,2.054746e+04,2.028746e+04,0
BTC-USD.Close,7.200174e+03,6.985470e+03,7.344884e+03,7.410657e+03,7.411317e+03,7.769219e+03,8.163692e+03,8.079863e+03,7.879071e+03,8.166554e+03,...,1.956701e+04,1.934557e+04,2.009586e+04,2.077044e+04,2.028584e+04,2.059535e+04,2.081848e+04,2.063560e+04,2.049577e+04,0
BTC-USD.Volume,1.856566e+10,2.080208e+10,2.811148e+10,1.844427e+10,1.972507e+10,2.327626e+10,2.876729e+10,3.167256e+10,2.404599e+10,2.871458e+10,...,2.212879e+10,3.020224e+10,4.776152e+10,5.889595e+10,4.962511e+10,4.399472e+10,4.036984e+10,3.148635e+10,4.566847e+10,0


In [100]:
#Training function

def generate_predictions(product):
    epsilonerror=1 #Notice that here we are adding the required error for the logaritmith, this is an hyperparameter of the model constructed in this script. 
    #Due to the high values of the data (the default value for epsilonerror is 1). 
    the_data=(df_product+epsilonerror).loc[product][:] 
    train_data_log = np.log(the_data[:-3])
    test_data_log = np.log(the_data[-3:-1])

    
#Definition and train of the model    

    arima_model_log = auto_arima(train_data_log, start_p=0, d=1, start_q=0, max_p=5, max_d=5, max_q=5, start_P=0, 
                        D=1, start_Q=0, max_P=5, max_D=5, max_Q=5, m=1, error_action='warn', trace=True, 
                        supress_warnings=True, stepwise=True)
    
#Contruction of the prediction data frame for the selected product

    prediction_log=arima_model_log.predict(n_periods=7, return_conf_int=True)
    
    
    product_column = np.full(5, product)

    missing_values_column = np.full(5, the_data["zeros"]-1)

    model_column = np.full(5, "arima_model_without_years")

    year_column=["2022-11", "2022-12", "2023-01", "2023-02", "2023-03"]

    prediction_column=np.exp(prediction_log[0])[2:].to_numpy()
    

#Conditional for the statistical information of the second parameter

    if len(arima_model_log.pvalues())==2:
        intercept_p_value_column=np.full(5, arima_model_log.pvalues()[0])
        intercept_std_error_column=np.full(5, arima_model_log.bse()[0])

    

#Statistical information of the parameter
    
    sigma2_p_value_column=np.full(5, arima_model_log.pvalues()[-1])

    sigma2_p_std_error_column=np.full(5, arima_model_log.bse()[-1])

    

#General statistic information

    AIC_column=np.full(5, arima_model_log.aic())
    
    min_conf_interval_column=np.exp(prediction_log[1][2:,0])
    max_conf_interval_column=np.exp(prediction_log[1][2:,1])
    
#Construction of the dataframe prediction
    
    columns_prediction=["product","model", "intercept_p_value", "sigma2_p_value", "intercept_std_error", 
                        "sigma2_p_std_error", "AIC", "min_conf_interval", "max_conf_interval", 
                        "year-month", "prediction", "missing_values"]
    
    
    df_prediction=pd.DataFrame(columns=columns_prediction)
    df_prediction["product"]=product_column
    df_prediction["model"]=model_column

    df_prediction["sigma2_p_value"]=sigma2_p_value_column
    df_prediction["sigma2_p_std_error"]=sigma2_p_std_error_column

    df_prediction["AIC"]=AIC_column    
    
    df_prediction["min_conf_interval"]=min_conf_interval_column
    df_prediction["max_conf_interval"]=min_conf_interval_column
    
    
    df_prediction["year-month"]=year_column
    df_prediction["prediction"]=prediction_column
    df_prediction["missing_values"]=missing_values_column

    if len(arima_model_log.pvalues())==2:
        df_prediction["intercept_p_value"]=intercept_p_value_column
        df_prediction["intercept_std_error"]=intercept_std_error_column
        
    
    
    return df_prediction


In [101]:


#Creation of an empty dataframe to fill up with the prediction values
columns_final_prediction=["product","model", "intercept_p_value", "sigma2_p_value", "intercept_std_error", 
                        "sigma2_p_std_error", "AIC", "min_conf_interval", "max_conf_interval", 
                        "year-month", "prediction", "missing_values"]

df_final_prediction=pd.DataFrame(columns=columns_final_prediction)


#Creation of the dataframe with all the prediction and their information
model_number=1
for product in df_product.index[:]:
    print("Training model number ", model_number)
    model_number=model_number+1
    new_prediction=generate_predictions(product)
    framespred= [df_final_prediction, new_prediction]
    df_final_prediction = pd.concat(framespred)
    


#Ccreating and saving the resulting dataframe

df_model_log_prediction=df_final_prediction.copy()
df_model_log_prediction=df_model_log_prediction.set_index(["product"])

df_model_log_prediction.to_csv('example_script_output.csv')


Training model number  1
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-3752.648, Time=0.18 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-3755.616, Time=0.69 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-3755.163, Time=0.19 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-3753.951, Time=0.12 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-3756.064, Time=0.45 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-3754.335, Time=0.26 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-3755.192, Time=0.44 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-3758.695, Time=1.53 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-3754.235, Time=0.51 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-3755.527, Time=0.42 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-3755.915, Time=1.46 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-3759.938, Time=0.31 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-3756.367, Time=0.12 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=-3756.817, Time=0.1

In [102]:

#visualization of the final dataframe
df_model_log_prediction.head(20)


,model,intercept_p_value,sigma2_p_value,intercept_std_error,sigma2_p_std_error,AIC,min_conf_interval,max_conf_interval,year-month,prediction,missing_values
product,,,,,,,,,,,
BTC-USD.Open,arima_model_without_years,NaN,0.0,NaN,0.000021,-3759.937838,18078.145275,18078.145275,2022-11,20564.173242,0.0
BTC-USD.Open,arima_model_without_years,NaN,0.0,NaN,0.000021,-3759.937838,17752.789695,17752.789695,2022-12,20580.279177,0.0
BTC-USD.Open,arima_model_without_years,NaN,0.0,NaN,0.000021,-3759.937838,17431.288566,17431.288566,2023-01,20568.333253,0.0
BTC-USD.Open,arima_model_without_years,NaN,0.0,NaN,0.000021,-3759.937838,17173.058967,17173.058967,2023-02,20577.192097,0.0
BTC-USD.Open,arima_model_without_years,NaN,0.0,NaN,0.000021,-3759.937838,16919.135837,16919.135837,2023-03,20570.621707,0.0
BTC-USD.High,arima_model_without_years,0.000007,0.0,0.029421,0.000019,-4121.939580,18624.886260,18624.886260,2022-11,21026.400245,0.0
BTC-USD.High,arima_model_without_years,0.000007,0.0,0.029421,0.000019,-4121.939580,18253.536065,18253.536065,2022-12,21026.400245,0.0
BTC-USD.High,arima_model_without_years,0.000007,0.0,0.029421,0.000019,-4121.939580,17934.915540,17934.915540,2023-01,21026.400245,0.0
BTC-USD.High,arima_model_without_years,0.000007,0.0,0.029421,0.000019,-4121.939580,17652.974719,17652.974719,2023-02,21026.400245,0.0
